In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.api import OLS
from scipy.optimize import minimize
from scipy.stats import t, norm, multivariate_normal
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

## Problem 1

In [14]:
np.random.seed(42)
num_simulations = 1000
sigma = 1  
P_t_minus_1 = 100  
returns = np.random.normal(0, sigma, num_simulations)

In [15]:
# Calculate prices for each of the return types
# Classical Brownian Motion
prices_Brownian = P_t_minus_1 + returns

# Arithmetic Return System
prices_Arithmetic = P_t_minus_1 * (1 + returns)

# Log Return or Geometric Brownian Motion
prices_Geometric = P_t_minus_1 * np.exp(returns)

# Calculate expected values and standard deviations
expected_value_Brownian = np.mean(prices_Brownian)
std_dev_Brownian = np.std(prices_Brownian)

expected_value_Arithmetic = np.mean(prices_Arithmetic)
std_dev_Arithmetic = np.std(prices_Arithmetic)

expected_value_Geometric = np.mean(prices_Geometric)
std_dev_Geometric = np.std(prices_Geometric)

In [16]:
print(f"Classical Brownian Motion: E[P_t] = {expected_value_Brownian:.2f}, SD[P_t] = {std_dev_Brownian:.2f}")
print(f"Arithmetic Return System: E[P_t] = {expected_value_Arithmetic:.2f}, SD[P_t] = {std_dev_Arithmetic:.2f}")
print(f"Log Return / Geometric Brownian Motion: E[P_t] = {expected_value_Geometric:.2f}, SD[P_t] = {std_dev_Geometric:.2f}")

Classical Brownian Motion: E[P_t] = 100.02, SD[P_t] = 0.98
Arithmetic Return System: E[P_t] = 101.93, SD[P_t] = 97.87
Log Return / Geometric Brownian Motion: E[P_t] = 168.23, SD[P_t] = 244.49


## Problem 2


In [26]:
df = pd.read_csv('DailyPrices.csv')

In [27]:
def return_calculate(prices, method="DISCRETE", date_column="date"):
    if date_column not in prices.columns:
        raise ValueError(f"dateColumn: {date_column} not in DataFrame: {prices.columns}")
    
    # Extract prices and compute returns
    p = prices.drop(columns=[date_column]).values
    n, m = p.shape
    p2 = np.empty((n - 1, m))
    
    for i in range(n - 1):
        for j in range(m):
            p2[i, j] = p[i + 1, j] / p[i, j]
    
    if method.upper() == "DISCRETE":
        p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError(f"method: {method} must be in (\"LOG\", \"DISCRETE\")")
    
    # Prepare the output DataFrame
    dates = prices[date_column].iloc[1:]
    out = pd.DataFrame(data=p2, columns=prices.columns.drop(date_column))
    out.insert(0, date_column, dates.values)
    
    return out


In [33]:
returns = return_calculate(df, "DISCRETE", "Date")
returns = returns.iloc[:-1]
returns

,Date,SPY,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,GOOG,BRK-B,...,CI,ETN,SLB,PGR,SCHW,LRCX,ZTS,C,BSX,AMT
0,2022-09-02,-0.010544,-0.013611,-0.016667,-0.002425,-0.020808,-0.017223,-0.025076,-0.016915,-0.016854,...,-0.001180,-0.010593,0.033107,-0.010428,-0.019242,-0.004236,-0.015244,0.001846,-0.012198,-0.026355
1,2022-09-06,-0.003773,-0.008215,-0.010974,-0.010980,-0.013336,-0.009643,0.015581,-0.011042,-0.003890,...,-0.004641,0.008449,-0.014118,0.000572,0.001848,-0.008019,-0.000892,-0.012695,-0.002717,0.013275
2,2022-09-07,0.017965,0.009254,0.019111,0.026723,0.018795,0.024717,0.033817,0.027912,0.016089,...,0.016652,0.020295,-0.008030,0.038537,0.018731,0.012279,0.022698,0.008503,0.026994,0.020930
3,2022-09-08,0.006536,-0.009618,0.001666,0.002626,0.020126,-0.009776,0.019598,-0.009595,0.008184,...,0.002448,0.013945,0.029951,0.015880,0.019083,0.016574,-0.011908,0.026116,0.029901,0.008362
4,2022-09-09,0.015535,0.018840,0.022977,0.026575,0.028377,0.020945,0.036023,0.021568,0.008576,...,0.007327,0.017244,0.038774,-0.004179,0.018863,0.026460,0.036721,0.015431,0.005385,-0.000306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,2023-09-15,-0.012048,-0.004154,-0.025037,-0.029920,-0.036879,-0.005069,-0.005977,-0.004964,-0.004438,...,-0.000318,-0.020302,-0.016558,-0.005947,-0.025770,-0.050759,-0.013626,-0.009968,-0.000378,-0.005191
260,2023-09-18,0.000586,0.016913,-0.003513,-0.002920,0.001503,0.005895,-0.033201,0.004772,0.006986,...,0.007485,0.006938,0.010399,0.013118,-0.006183,0.020125,-0.003329,-0.001639,0.001890,-0.003386
261,2023-09-19,-0.002074,0.006181,-0.001246,-0.016788,-0.010144,-0.001230,0.004599,-0.000936,0.000135,...,-0.002453,-0.013644,-0.012743,0.013589,-0.002247,-0.016519,0.012970,0.000938,0.000566,-0.012087
262,2023-09-20,-0.009193,-0.019992,-0.023977,-0.017002,-0.029435,-0.031150,-0.014672,-0.030541,-0.009879,...,0.009450,-0.006986,-0.010591,0.001544,-0.018361,-0.010062,-0.002748,-0.008903,0.020177,0.000282


In [34]:
returns['META'] = returns['META'] - returns['META'].mean()
returns['META']

0     -0.033234
1     -0.013858
2      0.008914
3      0.007657
4      0.040994
         ...   
259   -0.039358
260    0.004704
261    0.005574
262   -0.020456
263   -0.015903
Name: META, Length: 264, dtype: float64

In [51]:
# 1-Using a normal distribution
# assuming the confidence level is 0.95
alpha = 0.05
confidence_level = 0.95
meta_returns = returns['META']
mean = np.mean(meta_returns)
std_dev = np.std(meta_returns)

Z_score = norm.ppf(confidence_level)
var = Z_score * std_dev
var

0.054184407435059034

In [91]:
# 2-Using a normal distribution with an Exponentially Weighted variance (λ = 0.94)
lambda_param = 0.94
def calculate_ewma_variance(data, lambda_param):
    ewma_variance = np.zeros(len(data))
    ewma_variance[0] = data.var() 
    for t in range(1, len(data)):
        ewma_variance[t] = lambda_param * ewma_variance[t-1] + (1 - lambda_param) * (data.iloc[t-1] - data.iloc[:t].mean())**2
    return ewma_variance

variance_EW = calculate_ewma_variance(meta_returns, lambda_param)
std_EW = np.sqrt(variance_EW)
var_normal_EW = 1.65 * std_EW[-1]
var_normal_EW

0.030799203033063743

In [93]:
# 3-Using a MLE fitted T distribution.
params = t.fit(meta_returns)
VaR_t = t.ppf(0.95, *params)
VaR_t

0.043849319518122064

In [95]:
# 4-Using a fitted AR(1) model.

model = ARIMA(meta_returns, order=(1, 0, 0))
model_fitted = model.fit()
# Extract the residuals
residuals = model_fitted.resid

n_simulations = 1000
# Simulate future values based on residuals
simulated_values = np.random.choice(residuals, size=n_simulations)

# Calculate VaR from the simulated values at the 5% quantile
alpha = 0.05
VaR_ar1 = -np.quantile(simulated_values, alpha)
VaR_ar1

/Users/ada/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/ada/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/ada/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


0.04540614060753156

In [112]:
# 5-Using a Historic Simulation.

VaR_hist = meta_returns.sort_values().quantile(0.95)
VaR_hist

0.043957621172098844

## Problem 3

In [130]:
# Load data
prices = pd.read_csv('DailyPrices.csv').set_index('Date')
portfolio = pd.read_csv('Portfolio.csv')
lambda_EW = 0.94
alpha = 0.05  

# calculate EWMA covariance matrix
def calculate_ewma_cov(returns, lambda_EW):
    ewma_cov = returns.ewm(span=(2-lambda_EW)/lambda_EW, adjust=False).cov()
    return ewma_cov

# Calculate returns
returns = prices.pct_change().dropna()

# Calculate EWMA covariance matrix for the last date in the returns DataFrame
ewma_cov = calculate_ewma_cov(returns, lambda_EW).loc[returns.index[-1]]

portfolios = portfolio.groupby('Portfolio').apply(lambda x: dict(zip(x.Stock, x.Holding)))
portfolios

Portfolio
A    {'AAPL': 158, 'MSFT': 178, 'AMZN': 110, 'NVDA'...
B    {'CMCSA': 52, 'PFE': 90, 'LIN': 95, 'ORCL': 11...
C    {'GS': 52, 'BKNG': 131, 'MS': 134, 'MDT': 103,...
dtype: object

In [132]:
# Calculate latest prices for stocks
latest_prices = prices.iloc[-1]

# Initialize a dictionary to hold VaR values
VaRs = {}

# Calculate market values for each portfolio and total market value
total_market_value = 0

for name, holdings in portfolios.items():
    # Filter the returns for the current portfolio's stocks
    portfolio_returns = returns[list(holdings.keys())]

    # Calculate the EWMA covariance matrix for the portfolio
    portfolio_ewma_cov = calculate_ewma_cov(portfolio_returns, lambda_EW).loc[returns.index[-1], list(holdings.keys())]

    # Ensure ewma_covariance only includes relevant stocks
    relevant_cov_matrix = portfolio_ewma_cov.loc[list(holdings.keys()), list(holdings.keys())]

    # Calculate market values
    market_values = np.array([latest_prices[stock] * holdings[stock] for stock in holdings.keys()])
    portfolio_value = sum(market_values)
    total_market_value += portfolio_value

    # Calculate portfolio weights
    portfolio_weights = market_values / portfolio_value

    # Calculate portfolio variance using the weights and the relevant covariance matrix
    portfolio_var = np.dot(portfolio_weights.T, np.dot(relevant_cov_matrix, portfolio_weights))
    portfolio_std = np.sqrt(portfolio_var)

    # Calculate VaR for the portfolio
    VaR = -norm.ppf(alpha) * portfolio_std * portfolio_value
    VaRs[name] = VaR

# Output the results
for portfolio_name, VaR in VaRs.items():
    print(f"Portfolio {portfolio_name} VaR: ${VaR:.2f}")
print(f"Total VaR: ${sum(VaRs.values()):.2f}")

Portfolio A VaR: $25991.81
Portfolio B VaR: $10333.36
Portfolio C VaR: $21310.70
Total VaR: $57635.87


In [134]:
# log method
def calculate_ewma_cov(returns, lambda_EW):
    ewma_cov = returns.ewm(span=(2-lambda_EW)/lambda_EW, adjust=False).cov(pairwise=True)
    return ewma_cov.loc[returns.index[-1]]

# Calculate log returns for the entire 'prices' DataFrame
log_returns = np.log(prices / prices.shift(1)).dropna()

portfolio_dict = portfolio.groupby('Portfolio').apply(lambda x: dict(zip(x.Stock, x.Holding))).to_dict()

lambda_EW = 0.94
alpha = 0.05
VaRs = {}

latest_prices = prices.iloc[-1]  

for portfolio_name, holdings in portfolio_dict.items():
    # Filter log returns for the stocks present in the current portfolio
    portfolio_log_returns = log_returns[list(holdings.keys())]
    
    cov_matrix = calculate_ewma_cov(portfolio_log_returns, lambda_EW)
   
    total_portfolio_value = sum([latest_prices[stock] * holdings[stock] for stock in holdings])

    portfolio_weights = np.array([latest_prices[stock] * holdings[stock] for stock in holdings]) / total_portfolio_value

    portfolio_variance = np.dot(portfolio_weights.T, np.dot(cov_matrix.loc[holdings.keys(), holdings.keys()], portfolio_weights))

    portfolio_std = np.sqrt(portfolio_variance)

    VaR = norm.ppf(1 - alpha) * portfolio_std * total_portfolio_value
    VaRs[portfolio_name] = VaR

for portfolio_name, VaR in VaRs.items():
    print(f"Portfolio {portfolio_name} VaR: ${VaR:.2f}")
print(f"Total VaR: ${sum(VaRs.values()):.2f}")

Portfolio A VaR: $26234.16
Portfolio B VaR: $10475.39
Portfolio C VaR: $21449.62
Total VaR: $58159.17
